**Descrição do Projeto :**

1) No Google Colab, você precisa importar o conjunto de dados de produtos.

2) Execute operações de tokenização e preenchimento no conjunto de dados.

3) Crie um modelo de aprendizado de máquina a partir dele.

4) Preveja os sentimentos associados às avaliações dos clientes usando esse modelo.

5) Assim que isso funcionar no Google Colab, você pode baixar o modelo e o conjunto de dados e experimentá-lo também no editor do Visual Studio Code.

6) Se você quiser experimentá-lo também no editor do Visual Studio Code, o link para a solução do projeto é fornecido na última célula.

In [ ]:

!git clone https://github.com/ByjusBrazil/PRO_1-1_C115_ProductDataset

Cloning into 'PRO_1-1_C115_ProductDataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
import pandas as pd
dataframe = pd.read_excel("/content/PRO_1-1_C115_ProductDataset/updated_product_dataset.xlsx")
dataframe.head()

,Emoção,Texto
0,Positiva,ótimo pad amo ddr não quero pad de metal funci...
1,Positiva,excelente pad ótimo produto altamente responsi...
2,Positiva,incrível ótimo ddr pad funciona perfeitamente ...
3,Positiva,amo esperar deslizamento não muito feliz dia 1...
4,Positiva,preço de boa qualidade preço de boa qualidade ...


In [ ]:
dataframe["Emoção"].unique()

array(['Positiva', 'Negativa', 'Neutra'], dtype=object)

In [ ]:
encode_emotions = {"Neutra": 0, "Positiva": 1, "Negativa": 2}

In [ ]:
dataframe.replace(encode_emotions, inplace = True)
dataframe.head()

,Emoção,Texto
0,1,ótimo pad amo ddr não quero pad de metal funci...
1,1,excelente pad ótimo produto altamente responsi...
2,1,incrível ótimo ddr pad funciona perfeitamente ...
3,1,amo esperar deslizamento não muito feliz dia 1...
4,1,preço de boa qualidade preço de boa qualidade ...


In [ ]:
# Converta o Dataframe em uma lista de conjuntos de dados

training_sentences = []
training_labels = []

for i in range(len(dataframe)):
  sentence = dataframe.loc[i, "Texto"]
  training_sentences.append(sentence)
  label = dataframe.loc[i, "Emoção"]
  training_labels.append(label)


In [ ]:
training_sentences[10], training_labels[10]

('decepcionado lento tirar fotos', 0)

In [ ]:
# Tokenização: importe o Tokenizer do tensorflow

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

# Defina os parâmetros do Tokenizer
vocab_size = 40000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

# Crie um dicionário chamado word_index
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)

In [ ]:
# Defina os parâmetros das pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type='post'
max_length = 100
trunc_type='post'


training_padded = pad_sequences(training_sequences, maxlen=max_length, 
                                padding=padding_type, truncating=trunc_type)

In [ ]:
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D

model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(0.2),
        Conv1D(filters = 256, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),
        Conv1D(filters = 128, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),
        LSTM(128),
        Dense(128, activation = "relu"),
        Dropout(0.2),
        Dense(64, activation = "relu"),
        Dense(3, activation = "softmax")
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           640000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 256)           12544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 32, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 10, 128)          0         
 1D)                                                    

In [ ]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, verbose=2)

Epoch 1/30
1/1 - 3s - loss: 1.0989 - accuracy: 0.4667 - 3s/epoch - 3s/step
Epoch 2/30
1/1 - 0s - loss: 1.0994 - accuracy: 0.3333 - 40ms/epoch - 40ms/step
Epoch 3/30
1/1 - 0s - loss: 1.0998 - accuracy: 0.3333 - 40ms/epoch - 40ms/step
Epoch 4/30
1/1 - 0s - loss: 1.0999 - accuracy: 0.3333 - 38ms/epoch - 38ms/step
Epoch 5/30
1/1 - 0s - loss: 1.1012 - accuracy: 0.3333 - 39ms/epoch - 39ms/step
Epoch 6/30
1/1 - 0s - loss: 1.0976 - accuracy: 0.3333 - 38ms/epoch - 38ms/step
Epoch 7/30
1/1 - 0s - loss: 1.1006 - accuracy: 0.3333 - 37ms/epoch - 37ms/step
Epoch 8/30
1/1 - 0s - loss: 1.0982 - accuracy: 0.3333 - 38ms/epoch - 38ms/step
Epoch 9/30
1/1 - 0s - loss: 1.0997 - accuracy: 0.1333 - 38ms/epoch - 38ms/step
Epoch 10/30
1/1 - 0s - loss: 1.0990 - accuracy: 0.2667 - 43ms/epoch - 43ms/step
Epoch 11/30
1/1 - 0s - loss: 1.1016 - accuracy: 0.2000 - 39ms/epoch - 39ms/step
Epoch 12/30
1/1 - 0s - loss: 1.0982 - accuracy: 0.4667 - 37ms/epoch - 37ms/step
Epoch 13/30
1/1 - 0s - loss: 1.0993 - accuracy: 0.200

In [ ]:
model.save("Customer_Review_Text_Emotion.h5")

In [ ]:
sentence = ["Ótimo telefone para comprar. É uma compra incrível com ótima duração da bateria"]
sentence = ["Ótimo telefone. Bateria de longa duração e visual incrível"]

sentence = ["Pior fone de ouvido do mundo. O bluetooth também não está funcionando"]


sentence = ["Você pode procurar uma câmera melhor mas, por este valor, é OK para comprar"]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
result = model.predict(padded)
print(result)
label = np.argmax(result , axis=1)
label = int(label)

# encode_emotions = {"Neutra": 0, "Positiva": 1, "Negativa": 2}
for emotion in encode_emotions:
  if encode_emotions[emotion] == label:
    print(f"sentiment : {emotion} , label : {label}")

[[0.3343761 0.3428644 0.3227595]]
sentiment : Positiva , label : 1


**Link da solução do projeto [editor do Visual Studio Code]**

1) Link: https://github.com/ByjusBrazil/PRO_1-1_C115_ReferenciaProjetoCodigo

2) Consulte o documento do projeto para saber como executar este projeto usando o editor do Visual Studio Code.